# **Indicadors IST**

L'Índex Socioeconòmic Territorial (IST) es calcula tenint en compte indicadors sobre el nivell educatiu, renta mitja, atur, població immigrant, etc.

A més de l'IST per secció censal, l'IDESCAT ofereix també aquests indicadors utilitzats per calcular-lo.

A diferència del dataset de l'IST, aquests indicadors es troben agregats per districte, i en el cas del municipi de Barcelona, també en barris.

Aquest dataset integra els datasets d'indicadors per districte i d'indicadors per barris per aprofitar al màxim la desagregació més petita disponible.

* **Font**: Generalitat de Catalunya - IDESCAT
* **Nom**: Índex Socioeconòmic Territorial
* **URL**: https://www.idescat.cat/dades/obertes/ist
* **Periodicitat**: anual (actualitzat amb tres anys de retràs)
* **Agregació geogràfica**: per municipi (tota Catalunya) i per barris (només Barcelona)

De la URL, descarregar:
* Indicadors socioeconòmics. Valors | Municipis | 2015-202X
* Indicadors socioeconòmics. Valors | Barris de Barcelona | 2015-202X

In [2]:
# IMPORTS

import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Proj, transform
import numpy as np
import pandas as pd
import seaborn as sns
from shapely import wkt
import json
import datetime
import csv
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
FROM_YEAR = 2018
TO_YEAR = 2023

In [7]:
ROOT = "/content/drive/Shareddrives/GESTIÓ DE PROJECTES/X. Entregable/"

INDICADORS_IST_BARRI_CSV = ROOT + "cleaning/raw_data/indicadors_ist_barri.csv"
INDICADORS_IST_MUNICIPI_CSV = ROOT + "cleaning/raw_data/indicadors_ist_municipi.csv"
FULL_MAP_FILE = ROOT + "cleaning/clean_data/full_map.csv"

In [8]:
full_map = pd.read_csv(FULL_MAP_FILE, dtype={"CUSEC":str, "CUMUN":str, "CSEC":str, "CDIS":str, "CMUN":str, "CUDIS":str, "CUBAR":str, "CBAR":str})

In [9]:
indicadors_ist_barri = pd.read_csv(INDICADORS_IST_BARRI_CSV, delimiter=';')

indicadors_ist_barri = indicadors_ist_barri.drop(columns=['estat', 'municipi'], axis=1)
indicadors_ist_barri = indicadors_ist_barri.rename(columns={'barris de Barcelona': 'NBAR'})
indicadors_ist_barri["valor"] = indicadors_ist_barri["valor"].str.replace(",",".")
indicadors_ist_barri["valor"] = indicadors_ist_barri["valor"].astype(float)
indicadors_ist_barri = pd.pivot_table(indicadors_ist_barri, values='valor', index=['any', 'NBAR'], columns=['concepte']).reset_index().rename_axis(None, axis=1)
indicadors_ist_barri["NBAR"] = indicadors_ist_barri["NBAR"].replace({"Sant Gervasi-Galvany": "Sant Gervasi - Galvany",
                                              "Sant Gervasi-la Bonanova": "Sant Gervasi - la Bonanova",
                                              "Sants-Badal": "Sants - Badal",
                                              "el Poble Sec": "el Poble-sec"})
indicadors_ist_barri = indicadors_ist_barri[indicadors_ist_barri["NBAR"]!="total"]
indicadors_ist_barri = indicadors_ist_barri.merge(full_map[["NBAR", "CUSEC"]], on="NBAR", how="inner").drop(columns="NBAR")
indicadors_ist_barri = indicadors_ist_barri.reset_index(drop=True)

In [11]:
indicadors_ist_municipi = pd.read_csv(INDICADORS_IST_MUNICIPI_CSV, delimiter=';')

indicadors_ist_municipi = indicadors_ist_municipi.drop(columns=['estat'], axis=1)
indicadors_ist_municipi = indicadors_ist_municipi.rename(columns={'municipi': 'NMUN'})
indicadors_ist_municipi["valor"] = indicadors_ist_municipi["valor"].str.replace(",",".")
indicadors_ist_municipi["valor"] = indicadors_ist_municipi["valor"].astype(float)
indicadors_ist_municipi = pd.pivot_table(indicadors_ist_municipi, values='valor', index=['any', 'NMUN'], columns=['concepte']).reset_index().rename_axis(None, axis=1)
indicadors_ist_municipi["NMUN"] = indicadors_ist_municipi["NMUN"].replace({"Hospitalet de Llobregat, l'": "Hospitalet de Llobregat, L'"})

indicadors_ist_municipi = indicadors_ist_municipi[indicadors_ist_municipi["NMUN"]!="total"]
indicadors_ist_municipi = indicadors_ist_municipi[(indicadors_ist_municipi['NMUN']!="Barcelona")]
indicadors_ist_municipi = indicadors_ist_municipi.merge(full_map[["NMUN", "CUSEC"]], on="NMUN", how="inner").drop(columns="NMUN")
indicadors_ist_municipi = indicadors_ist_municipi.reset_index(drop=True)

In [12]:
indicadors_ist = pd.concat([indicadors_ist_municipi, indicadors_ist_barri])
indicadors_ist.head()

,any,estrangers de països de renda baixa o mitjana,població amb estudis baixos,població jove sense estudis postobligatoris,població ocupada,renda mitjana per persona,treballadors de baixa qualificació,CUSEC
0,2015,6.9,NaN,NaN,60.4,12678.0,11.5,0808901001
1,2015,6.9,NaN,NaN,60.4,12678.0,11.5,0808901002
2,2015,6.9,NaN,NaN,60.4,12678.0,11.5,0808901003
3,2015,6.9,NaN,NaN,60.4,12678.0,11.5,0808901004
4,2015,6.9,NaN,NaN,60.4,12678.0,11.5,0808901005


In [13]:
# EXPORT

CLEAN_DATA_PATH = ROOT + "/cleaning/clean_data/"
INDICADORS_IST_OUTPUT_FILE = "indicadors_ist.csv"

indicadors_ist.to_csv(CLEAN_DATA_PATH + INDICADORS_IST_OUTPUT_FILE, index=False)